In [6]:
import torch
from transformers import BartForConditionalGeneration, BartTokenizer

input_sentence = "They were there to enjoy us and they were there to pray for us."

model = BartForConditionalGeneration.from_pretrained('eugenesiow/bart-paraphrase')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
tokenizer = BartTokenizer.from_pretrained('eugenesiow/bart-paraphrase')



In [9]:
input_sentence = [
    "A purple elephant is flying in the sky, surrounded by pink clouds.",
    "The Eiffel Tower is standing tall in the middle of a dense forest.",
    "A group of penguins is sunbathing on a sandy beach.",
    "A rainbow-colored dinosaur is chasing a school bus on a city street.",
    "A mermaid is swimming in a fish tank filled with colorful tropical fish.",
    "A spaceship is landing on a snowy mountaintop, next to a cozy cabin.",
    "A giant cupcake is floating in the ocean, attracting seagulls.",
    "A giraffe is riding a bicycle in a crowded amusement park.",
    "A waterfall is flowing through a desert landscape with cacti and sand dunes."
]

batch = tokenizer(input_sentence, padding=True, truncation=True, return_tensors="pt").to(device)

import time
start = time.time()
translated = model.generate(**batch)
generated_sentences = tokenizer.batch_decode(translated, skip_special_tokens=True)
end = time.time()

print("Time taken in seconds: ", end - start)

print(generated_sentences)

/Users/riccardotedoldi/mambaforge/envs/ml/lib/python3.10/site-packages/transformers/generation/utils.py:1346: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Time taken in seconds:  4.072089195251465
['A purple elephant is flying in the sky, surrounded by pink clouds.', 'The Eiffel tower is tall in the middle of a dense forest.', 'A group of penguins is sunbathing on a sandy beach.', 'A rainbow-colored dinosaur is chasing a school bus on a city street.', 'A mermaid is swimming in a fish tank filled with colorful tropical fish.', 'A spaceship has landed on a snowy mountaintop, next to a cozy cabin.', 'A giant cupcake is floating in the ocean, attracting seagulls.', 'A giraffe is riding a bicycle in a crowded amusement park.', 'A waterfall flows through a desert landscape with cacti and sand dunes.']
